In [3]:
import importlib
import pathlib as pl
import sqlite3

from tqdm import tqdm

import grid_load.processing as glp

importlib.reload(glp)

conn = sqlite3.connect("local.db")

# files = list(pl.Path("../data").glob("*.csv"))
# for file in tqdm(files, total=len(files)):
#     df = glp.extract_data(file)
#     df = glp.transform_data(df)
#     if "Trafostation" in file.stem:
#         df.to_sql("grid_consumption", conn, if_exists="append", index=False)
#     elif "Wohnung" in file.stem:
#         user_id = int(file.stem.split("_")[0])
#         df["user_id"] = user_id
#         df.to_sql("user_consumption", conn, if_exists="append", index=False)

# conn.close()

In [4]:
import pandas as pd

sql = "Select * from grid_consumption"
df_lite = pd.read_sql(sql, conn)

In [5]:
df_lite

,Datum,Zeitstempel,ObisCode,Wert,Status,date_fmt,moving_avg,quantile,threshold,high_usage,diff_percentage,ts
0,01.01.2023 00:15:00 +01:00,1672528500,1.5.0,135.0,Normal,2022-12-31 23:15:00+00:00,1.000000,NaN,1.0,1,17956.000000,1.672528e+09
1,01.01.2023 00:30:00 +01:00,1672529400,1.5.0,120.6,Normal,2022-12-31 23:30:00+00:00,1.000000,NaN,1.0,1,14304.160000,1.672529e+09
2,01.01.2023 00:45:00 +01:00,1672530300,1.5.0,125.4,Normal,2022-12-31 23:45:00+00:00,1.000000,NaN,1.0,1,15475.360000,1.672530e+09
3,01.01.2023 01:00:00 +01:00,1672531200,1.5.0,141.6,Normal,2023-01-01 00:00:00+00:00,1.000000,NaN,1.0,1,19768.360000,1.672531e+09
4,01.01.2023 01:15:00 +01:00,1672532100,1.5.0,128.4,Normal,2023-01-01 00:15:00+00:00,1.000000,NaN,1.0,1,16230.760000,1.672532e+09
...,...,...,...,...,...,...,...,...,...,...,...,...
57973,11.09.2024 09:30:00 +02:00,1726039800,01.05.2000,28.2,Normal,2024-09-11 07:30:00+00:00,77.391071,86.7,86.7,0,-39.472318,1.726040e+09
57974,11.09.2024 09:45:00 +02:00,1726040700,01.05.2000,55.2,Normal,2024-09-11 07:45:00+00:00,77.393750,86.7,86.7,0,-11.444637,1.726041e+09
57975,11.09.2024 10:00:00 +02:00,1726041600,01.05.2000,61.8,Normal,2024-09-11 08:00:00+00:00,77.392857,86.7,86.7,0,-7.151211,1.726042e+09
57976,11.09.2024 10:15:00 +02:00,1726042500,01.05.2000,57.0,Normal,2024-09-11 08:15:00+00:00,77.369643,86.7,86.7,0,-10.174048,1.726042e+09


In [6]:
conn = sqlite3.connect("local.db")

start = 1724937300
end = 1726004700

sql = """
Select 
    * 
from 
    user_consumption 
where 
    user_id = :user_id
    AND ts >= :start
    AND ts <= :end
"""
df_user = pd.read_sql(
    sql,
    conn,
    params={
        "user_id": 220,
        "start": start,
        "end": end,
    },
)

sql_grid = """
SELECT
    *
FROM
    grid_consumption
WHERE
    ts >= :start
    AND ts <= :end
"""
df_grid = pd.read_sql(
    sql_grid,
    conn,
    params={
        "start": start,
        "end": end,
    },
)

corr = glp.correlation_matrix_stress_level(
    df_grid,
    df_user,
    window=24 * 4,
)

conn.close()